In [1]:
import pandas as pd
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import os
import gc
import re
import ast
import sys
import copy
import json
import time
import math
import string
import pickle
import random
import joblib
import itertools
import warnings
warnings.filterwarnings("ignore")
import scipy as sp
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import sys
sys.path.append("src")
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
from torch.optim import Adam, SGD, AdamW
from torch.utils.data import DataLoader, Dataset
import tokenizers
import transformers
print(f"tokenizers.__version__: {tokenizers.__version__}")
print(f"transformers.__version__: {transformers.__version__}")
from transformers import AutoTokenizer, AutoModel, AutoConfig
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
# %env TOKENIZERS_PARALLELISM=true
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import argparse
from utils import *

def get_result(oof_df,is_cla=False):
    labels = oof_df[CFG.target_cols].values
    preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
    if is_cla:
        score, scores = get_score_cla(labels, preds)
    else:
        score, scores = get_score(labels, preds)
    LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')

class CFG:
    wandb=False
    competition='Hate'
    debug=False
    apex=True
    print_freq=100
    num_workers=6
    model="microsoft/deberta-v3-large"#"Narrativaai/deberta-v3-small-finetuned-hate_speech18"#"microsoft/deberta-v3-base"
    gradient_checkpointing=True
    scheduler='cosine'
    batch_scheduler=True
    num_cycles=0.5
    num_warmup_steps=0
    epochs=5
    encoder_lr=2e-5
    decoder_lr=2e-5
    min_lr=1e-6
    eps=1e-6
    betas=(0.9, 0.999)
    batch_size=12
    max_len=512
    weight_decay=0.01
    gradient_accumulation_steps=1
    max_grad_norm=1000
    target_cols=['label_mean','HD', 'CV', 'VO','REL', 'RAE', 'SXO', 'GEN', 'IDL', 'NAT', 'POL', 'MPH', 'EX', 'IM']
    seed=42
    n_fold=4
    trn_fold=[0, 1, 2, 3]
    train=True
    pretrain_hate = False
    back_translation = False

tokenizers.__version__: 0.12.1
transformers.__version__: 4.21.2


In [2]:
train = pd.read_csv('./parler-hate-speech/GabHateCorpus_annotations_bt.csv')
train[CFG.target_cols] = train[CFG.target_cols] - train[CFG.target_cols].min()
train[CFG.target_cols] = train[CFG.target_cols] / train[CFG.target_cols].max()
Fold = MultilabelStratifiedKFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
for n, (train_index, val_index) in enumerate(Fold.split(train, train[CFG.target_cols+['id']])):
    train.loc[val_index, 'fold'] = int(n)
train['fold'] = train['fold'].astype(int)

In [3]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model)
CFG.tokenizer = tokenizer

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
lengths = []
tk0 = tqdm(train['text'].fillna("").values, total=len(train))
for text in tk0:
    length = len(tokenizer(text, add_special_tokens=False)['input_ids'])
    lengths.append(length)
CFG.max_len = min(509,max(lengths)) + 3 # cls & sep & sep

  0%|          | 0/27553 [00:00<?, ?it/s]

In [5]:
model = CustomModel(CFG, config_path=None, pretrained=True,LOGGER = get_logger(filename="temp"))

DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.21.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

Some weights of the model checkpoint at microsoft/deberta-v3-large were not used when initializing DebertaV2Mo

In [6]:
class TestDataset(Dataset):
    def __init__(self, cfg, df,text_col = "text"):
        self.cfg = cfg
        df = df.fillna("-9999")
        self.df = df
        self.texts = df[text_col].values
        
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        texts = self.texts[item]
        inputs = prepare_input(self.cfg, texts)
        return inputs

In [7]:
test_dataset_en = TestDataset(CFG,train,text_col = "text")
test_loader_en = DataLoader(test_dataset_en,
                        batch_size=CFG.batch_size * 2,
                        shuffle=False,
                        num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

test_dataset_de = TestDataset(CFG,train,text_col = "BackTranslation_de")
test_loader_de = DataLoader(test_dataset_de,
                        batch_size=CFG.batch_size * 2,
                        shuffle=False,
                        num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

test_dataset_fr = TestDataset(CFG,train,text_col = "BackTranslation_fr")
test_loader_fr = DataLoader(test_dataset_fr,
                        batch_size=CFG.batch_size * 2,
                        shuffle=False,
                        num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

test_dataset_es = TestDataset(CFG,train,text_col = "BackTranslation_es")
test_loader_es = DataLoader(test_dataset_es,
                        batch_size=CFG.batch_size * 2,
                        shuffle=False,
                        num_workers=CFG.num_workers, pin_memory=True, drop_last=False)

In [ ]:
from tqdm.auto import tqdm
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score,recall_score,roc_auc_score,accuracy_score
import glob
TH_FROM_BASE = 0.3
W = 0.75
MODEL = 'microsoft.deberta-v3-large'
EXP = 'Gab_backtranslate_cla'
CFG.model = 'microsoft/deberta-v3-large'
model = CustomModel(CFG, config_path=None, pretrained=True,LOGGER = get_logger(filename="temp"))
all_df = []
for fold in tqdm([0,1,2,3]):
    state = torch.load(glob.glob(f"./{EXP}/{MODEL}/*{fold}_best.pth")[0])
    model.load_state_dict(state["model"])
    model.eval()
    model = model.to(device)
    results_score = []
    results_emb = []
    for loader in tqdm([test_loader_en,test_loader_de,test_loader_fr,test_loader_es]):
        lg_embeddings = []
        lg_score = []
        for step, (inputs) in enumerate(loader):
            inputs = collate(inputs)
            for k, v in inputs.items():
                inputs[k] = v.to(device)
            with torch.no_grad():
                sentence_embeddings = model.feature(inputs)
                lg_embeddings.append(sentence_embeddings.to('cpu').numpy())
                y_preds = model(inputs)
                lg_score.append(y_preds.sigmoid().to('cpu').numpy())
        lg_score = np.concatenate(lg_score)
        lg_embeddings = np.concatenate(lg_embeddings)
        results_score.append(lg_score)
        results_emb.append(lg_embeddings)
    results_score = np.array(results_score)
    RES = []
    for t in range(results_score.shape[2]):
        results = []
        for i in range(len(results_score[0])):
            base = results_score[0][i][t]
            l = []
            for row in range(1,4):
                if abs(results_score[row][i][t] - base) < TH_FROM_BASE:
                    l.append(results_score[row][i][t])
            if len(l) ==0:
                results.append(base)
            else:
                results.append(W*base+(1-W)*np.mean(l))
        RES.append(results)
    df_res = pd.DataFrame(RES).T
    df_res.columns = ["pred_"+row for row in CFG.target_cols]
    T = pd.concat([train,df_res],axis=1)
    val = T[T['fold']==fold]
    all_df.append(val)
results_tta = pd.concat(all_df)

DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "attention_dropout": 0.0,
  "attention_probs_dropout_prob": 0.0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 1024,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "transformers_version": "4.21.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

DebertaV2Config {
  "_name_or_path": "microsoft/deberta-v3-large",
  "attention_dropout": 0.0,
  "attention_pr

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
results_tta

In [ ]:
results_tta.to_pickle(f"./{EXP}/{MODEL}/tta.pkl")

In [ ]:
# if True: 
#     results_score = np.array(results_score)
#     RES = []
#     for t in range(results_score.shape[2]):
#         results = []
#         for i in range(len(results_score[0])):
#             base = results_score[0][i][t]
#             l = []
#             for row in range(1,4):
#                 if abs(results_score[row][i][t] - base) < TH_FROM_BASE:
#                     l.append(results_score[row][i][t])
#             if len(l) ==0:
#                 results.append(base)
#             else:
#                 results.append(W*base+(1-W)*np.mean(l))
#         RES.append(results)
# df_res = pd.DataFrame(RES).T
# df_res.columns = ["pred_"+row for row in CFG.target_cols]
# T = pd.concat([train,df_res],axis=1)
# val = T[T['fold']==fold]

In [172]:
tta_res = []
label = []
for fold in [0,1,2,3]:
    val = train[train['fold']==fold]
    s = list(np.array(val[f'tta_score_fold_{fold}']).reshape(-1))
    l = list(np.array(val[f'label_mean']).reshape(-1))
    tta_res = tta_res + s
    label = label + l

In [182]:
def fix(row):
    return row[0]
train['tta_score_fold_0'] = train['tta_score_fold_0'].apply(fix)
train['tta_score_fold_1'] = train['tta_score_fold_1'].apply(fix)
train['tta_score_fold_2'] = train['tta_score_fold_2'].apply(fix)
train['tta_score_fold_3'] = train['tta_score_fold_3'].apply(fix)
train.to_pickle(f"./{EXP}/{MODEL}/tta.pkl")

In [177]:
res = np.sqrt(mean_squared_error(label,tta_res))*4    
res

0.8099878448924156

In [206]:
train2 = [pd.read_pickle(row).sort_values("id").reset_index(drop=True) for row in glob.glob("./*/*/*tta*")]

In [207]:
tta_res = []
label = []
for fold in [0,1,2,3]:
    val0 = train2[0][train2[0]['fold']==fold]
    val1 = train2[1][train2[1]['fold']==fold]
    val2 = train2[2][train2[2]['fold']==fold]
    s = list(np.array(val0[f'tta_score_fold_{fold}']).reshape(-1) + np.array(val1[f'tta_score_fold_{fold}']).reshape(-1) + np.array(val2[f'tta_score_fold_{fold}']).reshape(-1))
    s = np.array(s)/len(train2)
    l = list(np.array(val0[f'label_mean']).reshape(-1))
    tta_res = tta_res + list(s)
    label = label + l

In [208]:
res_tta = np.sqrt(mean_squared_error(label,np.array(tta_res)))*4    
res_tta

0.7754399632686458

In [258]:
pkl = [pd.read_pickle(row).sort_values("id").reset_index(drop=True) for row in glob.glob("*/*/*.pkl") if "_reg" in row and "tta" not in row and "large" in row]

In [259]:
s2 = [row['pred_label_mean'].values for row in pkl]

In [260]:
tta_res = []
label = []
for fold in [0,1,2,3]:
    val0 = train2[0][train2[0]['fold']==fold]
    val1 = train2[1][train2[1]['fold']==fold]
    val2 = train2[2][train2[2]['fold']==fold]
    s2 = [row[row['fold']==fold]['pred_label_mean'].values for row in pkl] + [row[row['fold']==fold]['pred_label_mean'].values for row in pkl]
    s2.append(np.array(val0[f'tta_score_fold_{fold}']).reshape(-1))
    s2.append(np.array(val1[f'tta_score_fold_{fold}']).reshape(-1))
    s2.append(np.array(val2[f'tta_score_fold_{fold}']).reshape(-1))
    s2 = np.mean(s2,axis=0)
    l = list(np.array(val0[f'label_mean']).reshape(-1))
    tta_res = tta_res + list(s2)
    label = label + l

In [261]:
res_tta_ensemble = np.sqrt(mean_squared_error(label,np.array(tta_res)))*4    
res_tta_ensemble

0.7632241102739699